In [97]:
from loader import *
import util

In [98]:
# Setup plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)

In [99]:
# Various imports
import pandas as pd
import numpy as np
import random

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers

In [100]:
TRAIN_AUTOENCODER = 0 
TRAIN_NET = 1

TOTAL_AE = 250000
TOTAL_MLP = 750000

BS_AE = 20
BS_MLP = 50
EPOCHS_AE = 50 
EPOCHS_MLP = 201 
RATE_AE = 0.005
DECAY_AE = 0.98
RATE_MLP = 0.005
DECAY_MLP = 0.98

BIAS = 0.15

N_INPUT = 769 
ENCODING_1 = 600 
ENCODING_2 = 400 
ENCODING_3 = 200
ENCODING_4 = 100

HIDDEN_1 = 200
HIDDEN_2 = 400 
HIDDEN_3 = 200
HIDDEN_4 = 100 
N_OUT = 2

VOLUME_SIZE = 25000

export_path = 'net/exports'

#Get the data from the game files
validation_test, validation_test_l = getTest(N_INPUT, 40, 44)
whiteWins, blackWins = getTrain(N_INPUT, TOTAL_MLP, VOLUME_SIZE)

Loading batch number 0
Loading batch number 1
Loading batch number 2
Loading batch number 3
Loading batch number 4
Loading batch number 5
Loading batch number 6
Loading batch number 7
Loading batch number 8
Loading batch number 9
Loading batch number 10
Loading batch number 11
Loading batch number 12
Loading batch number 13
Loading batch number 14
Loading batch number 15
Loading batch number 16
Loading batch number 17
Loading batch number 18
Loading batch number 19
Loading batch number 20
Loading batch number 21
Loading batch number 22
Loading batch number 23
Loading batch number 24
Loading batch number 25
Loading batch number 26
Loading batch number 27
Loading batch number 28
Loading batch number 29


In [101]:
# Creating the siamese autoencoders. SHAPE IS OFF, only position and player to move, not other data
input_shape = blackWins[0].shape
print("Input Shape:", input_shape)

# Layer 1
Pos2Vec = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layers.Dense(600, activation='relu', input_shape=input_shape),
    layers.Dense(769, activation='sigmoid'),
])

Input Shape: (769,)


In [103]:
from keras.optimizers import schedules

print("Defining Learning Rate")
# Defining a decaying learning rate
decay_optimizer = schedules.ExponentialDecay(
    initial_learning_rate=0.005,
    decay_steps= sample_size / 200,
    decay_rate=0.98,
    name=None)
    
# Compiling model
print("Compiling")
Pos2Vec.compile(
    optimizer=decay_optimizer,
    loss='mean_squared_error',
)

ImportError: cannot import name 'schedules' from 'keras.optimizers' (/Users/curtisbucher/.pyenv/versions/3.9.4/envs/neural-net/lib/python3.9/site-packages/keras/optimizers.py)

In [ ]:
sample_size

10000

In [ ]:
# whiteWins, blackWins
print("Generating random sample")
sample_size = 10000#1000000
test_size= 1000#1000
white_sample = whiteWins[np.random.randint(whiteWins.shape[0], size=sample_size), :]
black_sample = blackWins[np.random.randint(blackWins.shape[0], size=sample_size), :]

white_test = whiteWins[np.random.randint(whiteWins.shape[0], size=test_size), :]
black_test = blackWins[np.random.randint(blackWins.shape[0], size=test_size), :]

autoencoder_train = np.concatenate([white_sample, black_sample])
autoencoder_test = np.concatenate([white_test, black_test])

early_stopping = keras.callbacks.EarlyStopping(
    patience=500,
    min_delta=0.0001,
    restore_best_weights=True,
)

print("Training")
history = Pos2Vec.fit(
    autoencoder_train, autoencoder_train,
    validation_data=(autoencoder_test, autoencoder_test),
    batch_size=None,
    epochs=10, #200
    callbacks=[early_stopping],
    verbose=1,
)

Generating random sample
Defining Learning Rate
Training


TypeError: fit() got an unexpected keyword argument 'optimizer'

In [ ]:
# Training the 400 node layer
layer_1 = Pos2Vec.layers[0] # 769 - 600nodes
layer_2 = Pos2Vec.layers[1] # 600 - 769nodes

layer_1.trainable = False
layer_2.trainable = False

Pos2Vec_2 = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layer_1,
    layers.Dense(400, activation='relu'),
    layers.Dense(600, activation='relu'),
    layer_2,
])

In [ ]:
# Compiling model
Pos2Vec_2.compile(
    optimizer=decay_optimizer,
    loss='mean_squared_error',
)

In [ ]:
print("Training")
history = Pos2Vec_2.fit(
    autoencoder_train, autoencoder_train,
    validation_data=(autoencoder_test, autoencoder_test),
    batch_size=None,
    epochs=10, #200
    callbacks=[early_stopping],
    verbose=1,
)

Training
Epoch 1/10
625/625 [==============================] - 9s 14ms/step - loss: 0.0125 - val_loss: 0.0080
Epoch 2/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0081 - val_loss: 0.0077
Epoch 3/10
625/625 [==============================] - 8s 13ms/step - loss: 0.0079 - val_loss: 0.0075
Epoch 4/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0078 - val_loss: 0.0076
Epoch 5/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0078 - val_loss: 0.0075
Epoch 6/10
625/625 [==============================] - 7s 12ms/step - loss: 0.0078 - val_loss: 0.0076
Epoch 7/10
625/625 [==============================] - 8s 12ms/step - loss: 0.0077 - val_loss: 0.0077
Epoch 8/10
625/625 [==============================] - 8s 12ms/step - loss: 0.0078 - val_loss: 0.0078
Epoch 9/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0078 - val_loss: 0.0077
Epoch 10/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0077

In [ ]:
# Training the 400 node layer
layer_3 = Pos2Vec_2.layers[1] # 600 - 400nodes
layer_4 = Pos2Vec_2.layers[2] # 400 - 600nodes

layer_3.trainable = False
layer_4.trainable = False

Pos2Vec_3 = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layer_1,
    layer_3,
    layers.Dense(200, activation='relu'),
    layers.Dense(400, activation='relu'),
    layer_4,
    layer_2,
])

In [ ]:
# Compiling model
Pos2Vec_3.compile(
    optimizer=decay_optimizer,
    loss='mean_squared_error',
)

In [ ]:
print("Training")
history = Pos2Vec_3.fit(
    autoencoder_train, autoencoder_train,
    validation_data=(autoencoder_test, autoencoder_test),
    batch_size=None,
    epochs=10, #200
    callbacks=[early_stopping],
    verbose=1,
)

Training
Epoch 1/10
625/625 [==============================] - 7s 10ms/step - loss: 0.0115 - val_loss: 0.0088
Epoch 2/10
625/625 [==============================] - 5s 9ms/step - loss: 0.0088 - val_loss: 0.0085
Epoch 3/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0087 - val_loss: 0.0085
Epoch 4/10
625/625 [==============================] - 6s 10ms/step - loss: 0.0087 - val_loss: 0.0085
Epoch 5/10
625/625 [==============================] - 6s 10ms/step - loss: 0.0087 - val_loss: 0.0085
Epoch 6/10
625/625 [==============================] - 6s 10ms/step - loss: 0.0087 - val_loss: 0.0085
Epoch 7/10
625/625 [==============================] - 6s 10ms/step - loss: 0.0087 - val_loss: 0.0085
Epoch 8/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0087 - val_loss: 0.0085
Epoch 9/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0087 - val_loss: 0.0086
Epoch 10/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0087 

In [ ]:
# Training the 400 node layer
layer_5 = Pos2Vec_3.layers[2] # 400 - 200nodes
layer_6 = Pos2Vec_3.layers[3] # 200 - 400nodes

layer_5.trainable = False
layer_6.trainable = False

Pos2Vec_4 = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layer_1,
    layer_3,
    layer_5,
    layers.Dense(100, activation='relu'),
    layers.Dense(200, activation='relu'),
    layer_6,
    layer_4,
    layer_2,
])

In [ ]:

# Compiling model
Pos2Vec_4.compile(
    optimizer=decay_optimizer,
    loss='mean_squared_error',
)

In [ ]:
print("Training")
history = Pos2Vec_4.fit(
    autoencoder_train, autoencoder_train,
    validation_data=(autoencoder_test, autoencoder_test),
    batch_size=None,
    epochs=10, #200
    callbacks=[early_stopping],
    verbose=1,
)

Training
Epoch 1/10
625/625 [==============================] - 7s 10ms/step - loss: 0.0177 - val_loss: 0.0131
Epoch 2/10
625/625 [==============================] - 6s 10ms/step - loss: 0.0121 - val_loss: 0.0112
Epoch 3/10
625/625 [==============================] - 7s 12ms/step - loss: 0.0113 - val_loss: 0.0110
Epoch 4/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0112 - val_loss: 0.0110
Epoch 5/10
625/625 [==============================] - 8s 12ms/step - loss: 0.0111 - val_loss: 0.0109
Epoch 6/10
625/625 [==============================] - 9s 14ms/step - loss: 0.0111 - val_loss: 0.0109
Epoch 7/10
625/625 [==============================] - 8s 12ms/step - loss: 0.0111 - val_loss: 0.0108
Epoch 8/10
625/625 [==============================] - 7s 11ms/step - loss: 0.0111 - val_loss: 0.0109
Epoch 9/10
625/625 [==============================] - 8s 13ms/step - loss: 0.0111 - val_loss: 0.0108
Epoch 10/10
625/625 [==============================] - 8s 13ms/step - loss: 0.0109

In [ ]:
# Training the 400 node layer
layer_7 = Pos2Vec_4.layers[3] # 400 - 200nodes
layer_8 = Pos2Vec_4.layers[4] # 200 - 400nodes

layer_7.trainable = False
layer_8.trainable = False

Pos2Vec_Final = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layer_1,
    layer_3,
    layer_5,
    layer_7,
])


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 600)               462000    
                                                                 
 dense_14 (Dense)            (None, 400)               240400    
                                                                 
 dense_20 (Dense)            (None, 200)               80200     
                                                                 
 dense_22 (Dense)            (None, 100)               20100     
                                                                 
 dense_23 (Dense)            (None, 200)               20200     
                                                                 
 dense_21 (Dense)            (None, 400)               80400     
                                                                 
 dense_15 (Dense)            (None, 600)             